<a href="https://colab.research.google.com/github/PeterNaggschga/Letter-Variations-in-First-Names-IS/blob/main/LetterVariationsFirstNames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RadixTree
Implementation eines RadixTrees.

In [ ]:
class RadixTree:
    """contains a dict, which links to multiple RadixTree's. The key to another tree is a str and
    if a transition ends in a word, the isWord-variable is True
    """

    def __init__(self, isWord=False, transitions=None):
        if transitions is None:
            transitions = dict()
        self.isWord = isWord
        self.transitions = transitions

    def insertWord(self, word):
        """inserts a word into the tree

        Args:
            word (str): is the str, that gets inserted
        """
        for i in range(0, len(word)):
            # goes through 'tobi' with 'tobi', 'tob', 'to', 't'
            possibleTransition = word[:len(word) - i]
            if self.transitions.get(possibleTransition) is not None:
                child = self.transitions.get(possibleTransition)
                if possibleTransition == word:
                    child.isWord = True
                else:
                    child.insertWord(word[len(possibleTransition):])
                return

            for key in self.transitions.keys():
                if possibleTransition == key[:len(possibleTransition)]:
                    child = self.transitions.pop(key)
                    newDict = dict()
                    newDict[key[len(possibleTransition):]] = child
                    self.transitions[possibleTransition] = RadixTree(possibleTransition == word, newDict)
                    if possibleTransition != word:
                        self.transitions[possibleTransition].insertWord(word[len(possibleTransition):])
                    return

        self.transitions[word] = RadixTree(True, dict())

    def __strRecursive__(self, timesOfIndentation, lengthOfTransitionString):
        result = ""
        if self.isWord:
            result += "."
        keys = list(self.transitions.keys())
        keys.sort()
        for key in keys:
            recursiveResult = self.transitions[key].__strRecursive__(timesOfIndentation + lengthOfTransitionString,
                                                                     len(key))
            result += "\n" + (timesOfIndentation + lengthOfTransitionString) * "_" + key + recursiveResult
        return result

    def __str__(self):
        """generates a readable str, containing all class variables (the tree).

        Returns:
            str: the generated str
        """
        return self.__strRecursive__(0, 0)

    def getSimilarWordsOfSameLength(self, maximumDifferentLetters, word):
        """compares the given word with entries of the same length. Returns all of them with less or equal different letters than with maximumDifferentLetters described

        Args:
            maximumDifferentLetters (int): limits the amount of accepted different letters when comparing 2 words
            word (str): the given word

        Returns:
            list: returns a list of similar words with the same length. Does contain itself
        """
        if word == "":
            return [word] if self.isWord else []

        resultList = list()
        for key in self.transitions.keys():
            if len(key) > len(word):
                continue
            differences = 0
            for i in range(0, len(key)):
                differences += word[i] != key[i]
                if differences > maximumDifferentLetters:
                    break

            if differences > maximumDifferentLetters:
                continue
            resultTmp = self.transitions[key].getSimilarWordsOfSameLength(maximumDifferentLetters - differences,
                                                                          word[len(key):])
            if resultTmp:
                resultList.extend([key + tmp for tmp in resultTmp])
        return resultList


# RadixTreesByWordLength
Ein Container, der Wörter je nach Länge in unterschiedliche RadixTrees ablegt und ansonsten wie ein Tree agiert.

In [ ]:
class RadixTreesByWordLength:
    """contains a dict, which links to multiple RadixTree's, each storing words of the same length
    """

    def __init__(self):
        self.radixTrees = dict()

    def insertWord(self, word):
        """inserts a word into the tree

        Args:
            word (str): is the str, that gets inserted
        """
        length = len(word)
        if self.radixTrees.get(length) is None:
            self.radixTrees[length] = RadixTree()
        self.radixTrees[length].insertWord(word)

    def __str__(self):
        """generates a readable str, containing all class variables (the tree)

        Returns:
            str: the generated str
        """
        tmp = ""
        lengthStr = self.radixTrees.keys()
        sortedLengths = [int(lengths) for lengths in lengthStr]
        sortedLengths.sort()

        for length in sortedLengths:
            tmp += "RadixTree with words of length " + str(length) + ":"
            tmp += self.radixTrees[length].__str__()
            tmp += "\n"
        return tmp

    def getSimilarWordsOfSameLength(self, maximumDifferentLetters, word):
        """compares the given word with entries of the same length. Returns all of them with less or equal different letters than with maximumDifferentLetters described

        Args:
            maximumDifferentLetters (int): limits the amount of accepted different letters when comparing 2 words
            word (str): the given word

        Returns:
            list: returns a list of similar words with the same length. Does contain itself
        """
        length = len(word)
        if self.radixTrees.get(length) is None:
            return []
        return self.radixTrees[length].getSimilarWordsOfSameLength(maximumDifferentLetters, word)


# SubstituitionMatrix
Implementation einer BLOSUM-Substitutionsmatrix.

In [ ]:
class SubstitutionMatrix:
    """Stores letter-variations as a matrix. The matrix is a dict linking the first letter to a second dict, linking the second letter to its occurrence (int)
    """

    def __init__(self):
        self.letters = [
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
        self.matrix = dict()
        matrixTmp = dict()
        for letter in self.letters:
            self.matrix[letter] = dict()
            matrixTmp[letter] = 0
        for letter in self.letters:
            self.matrix[letter] = matrixTmp.copy()
        self.totalLetterTransitions = 0

    def addLetterTransition(self, fr, to):
        """adds the transistion from one letter to another in the matrix

        Args:
            fr (str): a single lower character of the alphabet
            to (str): a second single lower character of the alphabet

        Returns:
            bool: is False if one of the characters was not accepted, True if both were accepted
        """
        if fr not in self.letters or to not in self.letters:
            return False

        self.matrix[fr][to] += 1
        self.totalLetterTransitions += 1
        return True

    def getMatrixEntryInPercent(self, fr, to):
        """returns a specific matrix-entry in %

        Args:
            fr (str): a single lower character of the alphabet
            to (str): a second single lower character of the alphabet

        Returns:
            float: returns -1.0 if one of the given str's was not accepted, the matrix-entry in percent otherwise
        """
        if fr not in self.letters or to not in self.letters:
            return -1.0

        if self.totalLetterTransitions == 0:
            return 0.0
        return (self.matrix[fr][to] / self.totalLetterTransitions) * 100

    def __str__(self):
        """generates a readable str for the matrix

        Returns:
            str: the generated str
        """

        tmp = "Substitutionmatrix in %:\n"
        for to in self.matrix.keys():
            tmp += "\t" + to
        tmp += "\n"
        
        for fr in self.letters:
            tmp += fr
            for to in self.letters:
                strEntry = str(self.getMatrixEntryInPercent(fr, to))
                if len(strEntry) < 4:
                    strEntry = (4 - len(strEntry)) * "0" + strEntry
                tmp += "\t" + strEntry[:4]
            tmp += "\n"
        return tmp

    def addLetterVariationsToMatrix(self, name, similarNames):
        """retrieves differences of names with the original and inserts these letter-variations into the substitution-matrix

        Args:
            name (str): is the original name
            similarNames (list): list of similar names as str's
        """
        for similarName in similarNames:
            for i in range(len(name)):
                if name[i] != similarName[i]:
                    self.addLetterTransition(name[i], similarName[i])
                    

    # here is a simple way of storing and retrieveing the matrix in an external file
    """
    def save(self):
        f = open("backupFiles/SubstitutionMatrix.txt", "w")
        tmp = str(self.totalLetterTransitions) + ";"
        for key1 in self.matrix.keys():
            tmp += str(key1) + ":"
            for key2 in self.matrix[key1]:
                tmp += str(key2) + "-" + str(self.matrix[key1][key2]) + ","
            tmp = tmp[:-1] + "\n"
        f.write(tmp[:-1])
        f.close()
        
    def loadBackup(self):
        f = open("backupFiles/SubstitutionMatrix.txt", "r")
        backup = (f.read()).split(';')
        self.totalLetterTransitions = int(backup[0])
        backup = backup[1].split("\n")
        for entryStr in backup:
            key1 = entryStr[0]
            for entry in entryStr[2:].split(','):
                key2 = entry[0]
                self.matrix[key1][key2] = int(entry[2:])
        f.close()
    """


# Demo
Extraktion der Variationen in Vornamen.

In [ ]:
#import re
# install and import Entrez and Medline first
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    from Bio import Entrez, Medline
except:
    # One of these 2 lines should work
    # !pip install Bio
    install('Bio')
from Bio import Entrez, Medline


def getPapers(myQuery, maxPapers, myEmail="freytag64@gmail.com"):
    """retrieves some Papers from Pubmed

    Args:
        myQuery (str): is the given Query 
        maxPapers (int): is a limit of the number of papers, which will be retrieved
        myEmail (str, optional): an email. Defaults to "freytag64@gmail.com".

    Returns:
        list: papers as list of dictionarys containing abstract, authors, ...
    """
    # Get articles from PubMed
    Entrez.email = myEmail
    record = Entrez.read(Entrez.esearch(db="pubmed", term=myQuery, retmax=maxPapers))
    idlist = record["IdList"]
    print("\nThere are %d records for %s." % (len(idlist), myQuery.strip()))
    records = Medline.parse(Entrez.efetch(db="pubmed", id=idlist, rettype="medline", retmode="text"))
    return list(records)


def retrieveAllFirstNames(records):
    """takes list of papers (each is a dict) and extracts the first names as a combined list with a regular expression 

    Args:
        records (list): list of papers

    Returns:
        list: list of first names as str's
    """
    # retrieves first names from authors with regular expressions
    firstNameList = list()
    for record in filter(lambda x: 'FAU' in x, records):
        for fullName in record['FAU']:
            # since names are formatted like 'Lastname, Firstnames', split by ',' to get Firstnames
            firstName = fullName.split(',')[1].strip()
            names1 = list(filter(lambda x: len(x) > 1, firstName.split()))
            firstNameList.extend(names1)
            # print(fullName + " --> " + str(names1))
            
            # fixed: does not get name-pairs like "le Roux, Marlene F", since the lastname 'Roux' starts with ' ' too
            # fixed (see comment): does not really get name-pairs like "Something, A Mohammed", since A is just a single letter (it ignores A as a firstname)
            # a better one can be generated by using Multiple Sequence Alignment from the lectures. Names like Al-Abehd with '-' are just added as 'Al-Abehd' too.
            # sometimes accepts stuff like 'jr'
            
            """
            expression = r' ([a-zA-Z_-][a-zA-Z_-]+)'
            names2 = re.findall(expression, fullName)
            # firstNameList.extend(names2)            
            
            if names1 != names2:
              print(fullName + " --> " + str(names1))
              print(fullName + " --> " + str(names2))
            """

    return firstNameList


if __name__ == "__main__":
    maxPapers = 0  # limit the number of papers retrieved

    matrix = SubstitutionMatrix()  # saves letter-transitions in a matrix

    myQuery = "(\"2021/01/20\"[Date - Publication] : \"2021/01/20\"[Date - Publication])"
    records = getPapers(myQuery, maxPapers)


#Clustering
Clustern der Namen mit einer DBSCAN-ähnlichen Methode.

Unterschiede:
- zu große Cluster werden strenger erneut geclustert (maximale Größe: 1000 Namen)
- beim Prüfen, ob ein Knoten ein Kernknoten ist, zählen bereits erkundete Knoten nicht mehr
- Namen mit Länge 2 oder kürzer werden ignoriert

Ein Kernknoten braucht mindestens 6 unbesuchte Nachbarn um als Kernknoten zu gelten.

Eliminiert nicht erreichbare Knoten.

Knoten: einzelne Namen

Knotenübergänge (ungerichtet) zwischen Namen gleicher Länge mit maximaler Hamming-Distanz 1

In [1]:
def getNames() -> list:
    """reads names from external file (removes duplicates)

    Returns:
        list: list of firstnames
    """
    print("reads in names from external list")
    names = list()
    with open("2022FirstNames", "r") as myfile:
        try:
            names = list(set(myfile.read().split()))
        finally:
            myfile.close()

    print("names in total: " + str(len(names)))
    
    return names

def getClusters() -> list:
    """reads in clusters from external file

    Returns:
        list: is list of clusters (list of names)
    """
    print("reads in clusters from external list")
    clusters = list()
    with open("clusteredNames", "r") as myfile:
        try:
            clusterAsStrings = myfile.read().split("\n")
            for string in clusterAsStrings:
                clusters.append(string.split())
        finally:
            myfile.close()

    print("clusters in total: " + str(len(clusters)))
    
    return clusters
    

def Dbscan(names : list, min_samples : int, min_word_length : int, max_cluster_length : int) -> list:
    """clusters list of names with DBSCAN

    Args:
        names (list): list of names
        min_samples (int): the number of samples in a neighborhood for a point to be considered a core point
        min_word_length (int): only words with this length or longer are clustered
        max_cluster_length (int): maximum size of cluster. If found cluster are bigger, they will devided
    Returns:
        list: list of clusters (each cluster is a list of names)
    """
    
    tree = RadixTreesByWordLength()
    clusters = list()
    names = list(filter(lambda x : len(x) >= min_word_length, names))
    names.sort(reverse=True)
    
    tmp = list()
    for n in names:
        tmp.append(n.lower())
    names = tmp
    
    #construct the RadixTree
    for n in names:
        tree.insertWord(n)
    
    #clustering
    while len(names) > 0:
        name = names.pop()
        cluster = set()
        stack = [name]
        
        #fill cluster
        while len(stack) > 0:
            node = stack.pop()
            
            #check if name is a core-sample
            neighbors = tree.getSimilarWordsOfSameLength(1, node)
            #already found nodes are not considered as neighbors
            toBeRemoved = list()
            for n in neighbors:
                if n in cluster:
                    toBeRemoved.append(n)
                    continue
                for c in clusters:
                    if n in c:
                        toBeRemoved.append(n)
            for n in toBeRemoved:
                neighbors.remove(n) 
                
            #add new names to cluster
            if len(neighbors) >= min_samples:
                for n in filter(lambda x : x in names, neighbors):
                    names.remove(n)
                stack.extend(neighbors)
                cluster.update(set(neighbors))
        
        cluster = list(cluster)
        #add filled cluster
        if len(cluster) > 1 and len(cluster) <= max_cluster_length:
            clusters.append(cluster)
        #if a cluster is too big, cluster it again
        elif len(cluster) > max_cluster_length:
            smallerClusters = Dbscan(cluster, min_samples + 5, min_word_length, max_cluster_length)
            clusters.extend(smallerClusters)    

    return clusters
            

In [ ]:
names = getNames()

print("clustering (takes a while):")
clusters = Dbscan(names, 6, 3, 1000)

print(str(len(clusters)) + " different clusters")
i = 0
for c in clusters:
    i += len(c)
print(str(i) + " names clustered")

#save in file
with open("clusteredNames", "w+") as myfile:
    try:
        tmp = ""
        for cluster in clusters:
            for name in cluster:
                tmp += name + " "
            tmp += "\n"
        myfile.write(tmp)
    finally:
        myfile.close()